In [182]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# Import tables into dataframes from .txt files
df_outbound = pd.read_csv('Papper/Papper_Data/Papper_outbound.txt', sep='\t', index_col=0)
df_inbound = pd.read_csv('Papper/Papper_Data/Papper_inbound.txt', sep='\t', index_col=0)
df_CW_costs = pd.read_csv('Papper/Papper_Data/Papper_CW_costs.txt', sep='\t', index_col=0)
df_plant_costs_cap = pd.read_csv('Papper/Papper_Data/Papper_plant_costs_cap.txt', sep='\t', index_col=0)
df_demand = pd.read_csv('Papper/Papper_Data/Papper_demand.txt', sep='\t', index_col=0)

print(df_outbound.describe())


In [184]:
# Adjust any columns as necessary (here, RW5 column imported incorrectly, so was renamed)
df_outbound = df_outbound.rename(columns={' RW 5': 'RW 5'})

#Create list of plants, warehouses
cws = df_inbound.index.tolist()
plants = df_inbound.columns.tolist()
rws = df_outbound.columns.tolist()

In [ ]:
#create dictionaries for costs and constraints
outbound_dict = df_outbound.to_dict(orient='index')
inbound_dict = df_inbound.to_dict(orient='index')
variable_cost_CW = df_CW_costs['Variable costs'].to_dict()
fixed_cost_CW = df_CW_costs['Fixed costs'].to_dict()
variable_cost_plant = df_plant_costs_cap['Variable costs'].to_dict()
capacity_plant=df_plant_costs_cap['Capacity'].to_dict()
demand_dict = df_demand['Demand'].to_dict()

print(outbound_dict)
print(inbound_dict)
print(variable_cost_CW)
print(fixed_cost_CW)
print(variable_cost_plant)
print(capacity_plant)
print(demand_dict)
print(cws)
print(plants)
print(rws)

In [ ]:
# Model
model = gp.Model("Network Design")

# Decision variables
inbound = model.addVars(plants, cws, vtype=GRB.INTEGER, name=f"Inbound")
outbound = model.addVars(cws, rws, vtype=GRB.INTEGER, name=f"Outbound")
fixed = model.addVars(cws, vtype=GRB.BINARY, name=f"Binary")
inbound



[cw, rw]: This format is used for Gurobi decision variables (e.g., outbound[cw, rw]), where the variable directly uses both indices in a tuple.

[cw][rw]: This format is used for nested dictionaries (e.g., cw_rw_distances[cw][rw]), where the first key (cw) accesses a dictionary, and the second key (rw) accesses the value within that dictionary.


In [ ]:
# Objective function: Minimize total cost
model.setObjective(
    gp.quicksum(inbound[plant, cw] * inbound_dict[cw][plant] for plant in plants for cw in cws) + #inbound shipping
    #gp.quicksum(inbound[plant, cw] * inbound_dict[cw][plant] *0.3 for plant in plants for cw in cws) + #inbound shipping FOR PART 3, REDUCED COST
    gp.quicksum(outbound[cw, rw] * outbound_dict[cw][rw] for cw in cws for rw in rws) +   #outbound shipping
    gp.quicksum(inbound[plant, cw] * variable_cost_plant[plant] for plant in plants for cw in cws) +  #plant variable
    gp.quicksum(outbound[cw, rw] * variable_cost_CW[cw] for cw in cws for rw in rws) +  #CW variable
    gp.quicksum(fixed[cw] * fixed_cost_CW[cw] for cw in cws), #plant fixed
    GRB.MINIMIZE
)
model

In [188]:
# Set Constraints

# Demand constraint
#REMEMBER TO SET THIS AS == WHEN DEALING WITH LOS CONSTRAINT.
for rw in rws:  
    #model.addConstr(gp.quicksum(outbound[cw, rw] for cw in cws) >= demand_dict[rw], name=f"Demand_Constraint_{rw}")
    model.addConstr(gp.quicksum(outbound[cw, rw] for cw in cws) == demand_dict[rw], name=f"Demand_Constraint_{rw}")  #FOR LOS CONSTRAINT

# Capacity constraint
for plant in plants:  
    model.addConstr(gp.quicksum(inbound[plant, cw] for cw in cws) <= capacity_plant[plant], name=f"Capacity_Constraint_{plant}")

#Conservation of flow constraint
for cw in cws:
    model.addConstr(gp.quicksum(inbound[plant, cw] for plant in plants) == gp.quicksum(outbound[cw, rw] for rw in rws), name=f"Conservation_of_Flow{cw}")

#Linking constraint
M=1000000
for cw in cws:
    model.addConstr(gp.quicksum(outbound[cw, rw] for rw in rws) - fixed[cw] * M <= 0, name=f"Linking_Constraint_{cw}")


#Sum of Y's (open CWs)
y_low = 1
y_lim = 5
model.addConstr(gp.quicksum(fixed[cw] for cw in cws) >= y_low, name="Min_CWs_Open")
model.addConstr(gp.quicksum(fixed[cw] for cw in cws) <= y_lim, name="Max_CWs_Open")

#If Y is open, foce it to be in use
epsilon = 1  # Small value to ensure non-zero flow
for cw in cws:
    model.addConstr(gp.quicksum(outbound[cw, rw] for rw in rws) >= epsilon * fixed[cw], name=f"OutboundActive_{cw}")




In [ ]:
#Level of Service Constraints
#MAX DISTANCE FROM DC TO CUSTOMER -----------------------------------------------------------------
def Max_Distance_Condition(value):
    range = 60 #<-------------------- CHANGE THIS AS YOUR RANGE CONDITION
    if value > range:
        return 10000
    else:
        return value
    
# df_Max_Distance = df_outbound.applymap(Max_Distance_Condition)
# df_Max_Distance

#% CUSTOMER WITHIN RANGE OF DC -----------------------------------------------------------------
def Range_Condition(value, max_distance):
    if value > max_distance:
        return 0
    else:
        return 1

# df_Within_Range = df_outbound.applymap(Range_Condition)
# df_Within_Range

def apply_individual_los_constraints(model, df_outbound, outbound, demand_dict, rws, max_distance=70, percentage=0.6):
    # Create df_Within_Range inside this function using the updated max_distance
    df_Within_Range = df_outbound.applymap(lambda x: Range_Condition(x, max_distance))

    # Apply the LOS constraint using df_Within_Range
    for rw in rws:
        total_demand_rw = demand_dict[rw]  # Total demand for RW
        required_supply_within_range = percentage * total_demand_rw  # Required supply within range

        # Calculate supply from within range
        supply_from_within_range = gp.quicksum(outbound[cw, rw] * df_Within_Range.loc[cw, rw] for cw in cws)
        
        # Add constraint to the model ensuring at least the required percentage of demand is met from within the specified distance
        model.addConstr(supply_from_within_range >= required_supply_within_range, name=f"LOS_Constraint_{rw}")



#AGGREGATE LOS CONSTRAINT... ALL DEMAND TOTAL HAS TO BE 70% SATISFIED FROM WITHIN 70 MILES ----------------------->
##IMPORTANT#### ------------------------------------------------------> REMEMBE TO CHANGE DEMAND = IN THIS SCENARIO, OR DEMAND MAY BE OVERSUPPLIED. 
def apply_aggregate_los_constraint(model, df_outbound, outbound, demand_dict, cws, rws, max_distance=70, overall_percentage=0.7):
    # Create a DataFrame for within range based on max_distance
    df_Within_Range = df_outbound.applymap(lambda x: Range_Condition(x, max_distance))

    # Calculate the total demand across all RWs
    total_demand_all_rws = sum(demand_dict[rw] for rw in rws)

    # Calculate the total required supply within range for all RWs combined
    total_required_supply_within_range = overall_percentage * total_demand_all_rws

    # Sum the supplies from all CWs within 70 miles across all RWs and set the constraint
    model.addConstr(
        gp.quicksum(outbound[cw, rw] * df_Within_Range.loc[cw, rw] for cw in cws for rw in rws) >= total_required_supply_within_range,
        name="Aggregate_LOS_Constraint"
    )


def apply_maximum_distance_constraint(model, outbound, cws, rws, max_distance):
     '''
          Add constraints to the model to ensure deliveries only occur within the maximum allowed distance. 

     '''
     # Create a DataFrame for within range based on max_distance
     df_Within_Range = df_outbound.applymap(lambda x: Range_Condition(x, max_distance))
     for rw in rws:
        for cw in cws:
            # Add constraint: outbound[cw, rw] must be zero if the distance is not acceptable
            model.addConstr(outbound[cw, rw] * df_Within_Range.loc[cw, rw] == outbound[cw, rw], name=f"MaxDist_{cw}_{rw}")


#apply_maximum_distance_constraint(model, outbound, cws, rws, max_distance=1000)
apply_aggregate_los_constraint(model, df_outbound, outbound, demand_dict, cws, rws, max_distance=70, overall_percentage=0.6)
#apply_individual_los_constraints(model, df_outbound, outbound, demand_dict, rws, max_distance=80, percentage=0.6)



In [ ]:
model.optimize()

In [ ]:
if model.status == GRB.OPTIMAL:
    print(f"Status: Optimal")
    print(f"Total Cost: {model.objVal}\n")
    print(f"{'Retail Warehouse':<20} {'Demanded':<10} {'Delivered':<10}")
    print("-" * 40)

    for rw in rws:
        demanded = demand_dict[rw]
        delivered = sum(outbound[cw, rw].X for cw in cws if outbound[cw, rw].X > 0)  # Sum only non-zero deliveries
        print(f"{rw:<20} {demanded:<10} {delivered:<10.2f}")

    print("\nDecision Variable Values (non-zero):")
    print(f"{'Variable':<20} {'Value':<10}")
    print("-" * 30)
    
    for v in model.getVars():
        if v.X != 0:  # Print only non-zero variables
            print(f"{v.varName:<20} {v.X:<10.2f}")

else:
    print(f"Status: {model.status}")



In [ ]:
print('inbound shipping: ', gp.quicksum(inbound[plant, cw] * inbound_dict[cw][plant] for plant in plants for cw in cws).getValue())
print('outbound shipping: ', gp.quicksum(outbound[cw, rw] * outbound_dict[cw][rw] for cw in cws for rw in rws).getValue())
print('plant variable: ', gp.quicksum(inbound[plant, cw] * variable_cost_plant[plant] for plant in plants for cw in cws).getValue())
print('cw variable: ', gp.quicksum(outbound[cw, rw] * variable_cost_CW[cw] for cw in cws for rw in rws).getValue())
print('plant fixed: ', gp.quicksum(fixed[cw] * fixed_cost_CW[cw] for cw in cws).getValue())